In [1]:
# Data handling
import pandas as pd
import numpy as np

# Text processing
import nltk
import spacy
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import tweepy

# Data collection
import snscrape.modules.twitter as sntwitter

print("Completed Env Setup")

AttributeError: 'FileFinder' object has no attribute 'find_module'